In [15]:
import numpy as np
import pandas as pd

In [16]:
# метрика відстані між кластерами
def euclid_sqr(a, b):
    distance = np.sum(np.square(a-b), axis=1)
    return distance

In [17]:
# створення матриці близькості
def initialize_prox_mx(df):
    n = df.shape[0]
    objects = df.index
    indices = objects[1:]
    cols = objects[:objects.shape[0]-1]
    prox_mx = pd.DataFrame(np.zeros((n-1,n-1)), index=indices, columns=cols)
    # заповнення матриці близькості
    for obj in cols:
        curr = df.loc[obj]
        prox_mx[obj] = euclid_sqr(curr, df)
    # обираємо тільки нижньотрикутну матрицю
    prox_mx = prox_mx.where(np.tril(np.ones(prox_mx.shape)).astype(bool))
    return prox_mx

In [18]:
# функція злиття кластерів
def merge_clusters(df, prox_mx, classif_mx):
    # знаходимо мінімальну відстань з-поміж усіх попарних кластерів
    cluster_1 = prox_mx.min(axis=1).idxmin()
    cluster_2 = prox_mx.min(axis=0).idxmin()
    cluster_1_list, cluster_2_list = cluster_1.split(', '), cluster_2.split(', ')
    cluster_1_list = [cl.replace('(', '').replace(')', '') for cl in cluster_1_list]
    cluster_2_list = [cl.replace('(', '').replace(')', '') for cl in cluster_2_list]
    # таблиця для розрахунку центру кластера
    temp_df = pd.DataFrame(columns=features)
    for obj in cluster_1_list:
        temp_df.loc[obj] = data.loc[obj]
    for obj in cluster_2_list:
        temp_df.loc[obj] = data.loc[obj]
    cluster_center = temp_df.mean(axis=0)
    #for obj in cluster_1_list+cluster_2:
        
    # злиття об’єктів у кластери
    df = df.drop(labels=[cluster_1, cluster_2], axis=0, errors='ignore')
    df.loc[f"({cluster_1}, {cluster_2})"] = pd.Series(cluster_center)
    classif_mx.loc[len(classif_mx)] = [cluster_1_list[-1], cluster_2_list[-1], prox_mx.min().min(), cluster_2_list[-1]]
    return df, classif_mx

In [19]:
# фінальна функція створення матриці класифікації
def agg_hierarchial_clustering(df):
    classif_mx = pd.DataFrame(columns=['Об’єкт 1', 'Об’єкт 2', 'Відстань між кластерами', 'Нова назва'])
    while df.shape[0]!=1:
        prox_mx = initialize_prox_mx(df)
        df, classif_mx = merge_clusters(df, prox_mx, classif_mx)
    return classif_mx

In [24]:
# завантаження даних
data = pd.read_csv('https://hbiostat.org/data/repo/dmd.csv', index_col=0)
features = ['ck', 'h', 'carrier']
# обираємо 20 випадкових зразків
data = data[features].sample(n=20)
data.index = data.index.map(str)
data.to_csv('dataset.csv', sep=';')

In [25]:
df = data.copy()
df.head()

,ck,h,carrier
34,22.0,102.000000,0
57,74.0,80.390625,0
145,32.0,86.500000,0
139,34.0,92.000000,0
106,21.0,93.187500,0


In [26]:
classif_mx = agg_hierarchial_clustering(df)
classif_mx.to_csv('classif_mx.csv', sep=';', encoding='utf-8-sig')
classif_mx

,Об’єкт 1,Об’єкт 2,Відстань між кластерами,Нова назва
0,175,145,10.041260,145
1,93,34,12.250000,34
2,169,106,17.410156,106
3,145,139,31.877502,139
4,106,73,45.227539,73
5,34,46,57.595947,46
6,179,57,62.902588,57
7,157,18,65.000000,18
8,139,117,87.808404,117
9,117,26,99.583633,26
